In [3]:
%pip install sudachipy sudachidict_core

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.4 MB 1.9 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.4 MB 3.9 MB/s eta 0:00:01
   ------------------ --------------------- 0.6/1.4 MB 5.0 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.4 MB 5.5 MB/s eta 0:00:01
   ------------------------------------- -- 1.3/1.4 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.1 MB ? eta -:--:--
   ---------------------------------------- 0.4/72.1 MB 12.6 MB/s eta 0:00:06
   ---------------------------------------- 0.8/72.1 MB 9.6 MB/s eta 0:00:08
    --------------------------------------- 1.1/72.1 MB 8.8 MB/s eta 0:00:09
    --------------------------------------- 1.5/72.1 MB 8.7 MB/s eta 0:00:09
   - -------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import sudachipy
import numpy as np


In [57]:
text_file_n = 126
id_word_list = []
word_id_dict = {}

id_name_list = []
name_id_dict = {}
# <NULL>は遷移単語候補の終わりや、前の単語が存在しない場合の仮の単語を表す(「<NULL><NULL><SOS>今日……」のように)
# <SOS>は文章の始まりを表す
# <EOS>は文章の終わりを表す
markov_order = 2

word_transition_dict = []

sudachi_dict = sudachipy.Dictionary()
tokenizer = sudachi_dict.create()


In [58]:
def read_corpus_text(corpus_text: str, i):
    if corpus_text == '\n':
        return
    
    colon_index = corpus_text.find('：')

    name = corpus_text[:colon_index]
    
    if name[:3] in ['勝利時']:
        name = name[3:]
    
    save_name(name)

    main_text = corpus_text[colon_index + 1:]
    read_main_text(name, main_text)

def save_name(name: str):
    if not name in id_name_list:
        name_id_dict[name] = len(id_name_list)
        id_name_list.append(name)
        word_transition_dict.append({})

def read_main_text(name: str, main_text: str):
    morpheme_list = tokenizer.tokenize(main_text)
    name_id = name_id_dict[name]
    
    recent_word_list = []   
    recent_word_list = reset_recent_word_list()

    for morpheme in morpheme_list:
        word = morpheme.surface()

        if word in ('\n', '', ' ', '\t\n', '\t'):
            continue

        if word == '"':
            word = '”'

        save_word(word)
        save_word_transition(name_id, word, recent_word_list)

        recent_word_list = update_recent_word_list(word, recent_word_list)
    
    word = '<EOS>'
    save_word(word)
    save_word_transition(name_id, word, recent_word_list)

def reset_recent_word_list():
    recent_word_list = [0 for i in range(markov_order - 1)]
    recent_word_list.append(word_id_dict['<SOS>'])

    return recent_word_list

def update_recent_word_list(word, recent_word_list):
    recent_word_list[:-1] = recent_word_list[1:]
    recent_word_list[-1] = word_id_dict[word]

    return recent_word_list

def save_word_transition(name_id, word, recent_word_list):
    index = tuple(recent_word_list)

    if not index in word_transition_dict[name_id].keys():
        word_transition_dict[name_id][index] = []
    
    word_id = word_id_dict[word]

    if not word_id in word_transition_dict[name_id][index]:
        word_transition_dict[name_id][index].append(word_id)

def save_word(word):
    if not word in id_word_list:
        word_id_dict[word] = len(id_word_list)
        id_word_list.append(word)


In [64]:
def write_name_list():
    text_list = []

    for (name, id) in name_id_dict.items():
        text = ','.join((str(id), name)) + '\n'
        text_list.append(text)
    
    with open('./name_list.csv', 'w') as f:
        f.writelines(text_list)


def write_word_list():
    text_list = []

    for (word, id) in word_id_dict.items():
        text = ','.join((str(id), word)) + '\n'
        text_list.append(text)
    
    with open('./word_list.csv', 'w') as f:
        f.writelines(text_list)


def write_word_transition_dict(name_id: int):
    text_list = []
    sorted_word_transition = sorted(word_transition_dict[name_id].items(), key=lambda x: x[0])

    for (index, word_id_list) in sorted_word_transition:
        text = ','.join((str(i) for i in (*index, *sorted(word_id_list)))) + '\n'
        text_list.append(text)
    
    with open(f'./word_transition/word_transition_dict_{name_id}.csv', 'w') as f:
        f.writelines(text_list)


In [60]:
for word in ['<NULL>', '<SOS>', '<EOS>']:
    save_word(word)


In [61]:
text_file_n = 126

for i in range(text_file_n):
    with open(f'./dialogues/dialogue_{i}.txt') as f:
        corpus_text_list = f.readlines()
        for corpus_text in corpus_text_list:
            read_corpus_text(corpus_text, i)


In [65]:
write_name_list()
write_word_list()
write_word_transition_dict(0)


In [63]:
id_word_list[3186]

'及ん'